# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Answer Key

## Part A | RDBMS Databases and `SQLite`

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [3]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,2121978635,"829 Folsom St UNIT 906, San Francisco, CA",37781429,-122401860,0,...,557.0,sqft,NaN,None,2010.0
1,89239580,"690 Market St UNIT 1705, San Francisco, CA",37788246,-122403198,0,...,1050.0,sqft,NaN,None,2007.0
2,15131782,"401 Grand View Ave APT 3, San Francisco, CA",37752157,-122442356,0,...,937.0,sqft,NaN,None,1983.0
3,15179502,"250 Concord St, San Francisco, CA",37710141,-122442063,0,...,1574.0,sqft,1947.0,sqft,1959.0
4,52266124,"88 King St APT 317, San Francisco, CA",37780630,-122389635,0,...,1205.0,sqft,NaN,None,2000.0
5,2100994004,"409 Miguel St, San Francisco, CA",37736625,-122427008,0,...,400.0,sqft,NaN,None,NaN
6,15067755,"1250 Clay St APT 306, San Francisco, CA",37793575,-122413670,0,...,541.0,sqft,NaN,None,1950.0
7,15112556,"1819 26th Ave, San Francisco, CA",37753480,-122484273,0,...,1175.0,sqft,2996.0,sqft,1939.0
8,15133321,"365 Magellan Ave, San Francisco, CA",37744952,-122463110,0,...,3094.0,sqft,3920.0,sqft,1926.0
9,61288341,"710 Masonic Ave, San Francisco, CA",37774719,-122445912,0,...,1405.0,sqft,NaN,None,1914.0


### `SELECT`

In [4]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15165953,650000.00,$
1,80749447,1.15,$M
2,15155751,665000.00,$
3,15143887,2.10,$M
4,15117639,1.35,$M
...,...,...,...
995,69819708,731000.00,$
996,15076156,5.53,$M
997,119685619,625000.00,$
998,15113584,895000.00,$


### `WHERE`

In [5]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,80749447,1.15
1,15143887,2.10
2,15117639,1.35
3,80743040,1.48
4,15134909,1.09
...,...,...
570,2101028916,2.45
571,15091950,1.75
572,89237766,1.16
573,15197693,1.36


> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [6]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
420,119685503,0.750
421,69819708,0.731
422,119685619,0.625
423,15113584,0.895


### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [7]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `WITH`

In [8]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `JOIN`

In [9]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `WITH` (take 2)

In [10]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [11]:
pd.io.sql.read_sql(
'''
SELECT COUNT() AS count
    FROM properties
    GROUP BY beds
;
''', con = db)

,count
0,164
1,142
2,316
3,226
4,103
5,28
6,9
7,6
8,3
9,3


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [12]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

,mean_sale_price
0,1.739026
1,0.987656
2,1.420000
3,1.600000
4,1.223378
...,...
16,16.000000
17,0.999000
18,5.530000
19,13.100000


Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [13]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT AVG(normalized_sale_prices.sale_price) AS mean_sale_price
    FROM normalized_sale_prices
    JOIN properties
        ON properties.id = normalized_sale_prices.id
    WHERE properties.is_a_studio = 1
''', con = db)

,mean_sale_price
0,1.464034


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [14]:
pd.io.sql.read_sql(
'''
SELECT a.address AS address_a, b.address as address_b,
        a.id AS id_a, b.id as id_b
    FROM properties AS a
    JOIN properties AS b
        ON (a.id < b.id) AND
            (b.latitude = a.latitude) AND (b.longitude = a.longitude)
;
''', con = db)

,address_a,address_b,id_a,id_b
0,"88 King St APT 317, San Francisco, CA","88 King St APT 419, San Francisco, CA",52266124,53153575
1,"1177 California St APT 702, San Francisco, CA","1177 California St APT 501, San Francisco, CA",15068514,15068551
2,"1177 California St APT 702, San Francisco, CA","1177 California St APT 423, San Francisco, CA",15068514,15068580
3,"1177 California St APT 702, San Francisco, CA","1177 California St APT 304, San Francisco, CA",15068514,15068592
4,"400 Alemany Blvd APT 12, San Francisco, CA","400 Alemany Blvd APT 15, San Francisco, CA",61288494,61325457
...,...,...,...,...
103,"338 Spear St UNIT 5J, San Francisco, CA","338 Spear St UNIT 39A, San Francisco, CA",89238065,89240348
104,"301 Gough St APT 3, San Francisco, CA","301 Gough St # G, San Francisco, CA",15078419,15078423
105,"77 Dow Pl APT 500, San Francisco, CA","77 Dow Pl APT 507, San Francisco, CA",60825591,61288454
106,"765 Market St APT 27B, San Francisco, CA","765 Market St APT 31H, San Francisco, CA",63197318,63197354
